In [ ]:
import csv
from datetime import datetime, timedelta
import time
import numpy as np
from collections import defaultdict, namedtuple
import sys

Na prática, saber se é caso pai ou filho não interessa muito, pois não vou ver nesse nível

# Importando dados e  criando variáveis auxiliares

In [ ]:
demanda = {}
formato = '%Y-%m-%d %H:%M:%S'
with open('demanda.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    
    _ = next(reader)
    
    for row in reader:
        numerocasofilho = '{}_{}'.format(row[0], row[2].split(' ')[1])
        numerocaso = row[0]
        skill = row[2]
        #não interessa a data de entrada
        data_final = datetime.strptime(row[4],formato)
        demanda[numerocasofilho] = [numerocaso,skill,data_final]

# criando variáveis auxiliares
demanda_etapa =  {}
demanda_prazo =  {}
for key,value in demanda.items():
    demanda_etapa[key] = value[1]
    demanda_prazo[key] = value[2]

# Definindo funcionais para trabalho

In [ ]:
class grupo(object):
    
    def __init__(self, current_time = datetime.today()):
        self.__prioridades = defaultdict(list)
        self.__cdatetime = current_time
        self.__ocupada = {}
    
    def funcional(self,funcional, skills_tma):
        
        #para controlar para cada funcional se está ocupada
        self.__ocupada[ funcional]= 0 #guarda o minuto em que volta a estar disponível  
        
        for prioridade, skill in enumerate(skills_tma.keys()):
            #pela ordem cada skill vai para uma prioridade (proeficiência)
            self.__prioridades[prioridade].append({skill: [funcional, skills_tma[skill]]})
                
    def atendimento(self, minutos, demanda_etapa, demanda_prazo, individual):
        '''Recebe o minuto de planejamento e indivíduo gerado (demandas ordenadas)'''
        
        #mandar dentro de cada etapa as rds em ordem de atendimento acelera as próximas etapas
        etapa_rd = defaultdict(list)
        _ = {etapa_rd[demanda_etapa[rd]].append(rd) for rd in individual}
        
        #for key in etapa_rd:
            #print('Quantidade{}--{}'.format(key, len(etapa_rd[key])))
        
        sla_rd = 0
        
        #1. A simulação deve consumir as demandas dentro de uma janela de tempo
        for minuto in range(1,minutos+1):
            #print('minuto {}'.format(minuto))
            
            #2. Por prioridade (proeficiência) as demandas devem ser alocadas
            for proeficiencia, skills in self.__prioridades.items():
                
                #3. Acessando os skills da proeficiência
                for skill in skills:
                    
                    #4. Verificar se existe demanda para o skill acessado
                    skill_acessado = list(skill.keys())[0]
                    if skill_acessado in etapa_rd.keys():
                        
                        #5. Verificar se funcional está disponível
                        funcional, tma = skill[skill_acessado][0], skill[skill_acessado][1]
                        
                        if self.__ocupada[funcional] <= minuto:
                            #verificado que existe demanda e a funcional está disponível
                            
                            #retirando seguindo a ordem
                            rd = etapa_rd[skill_acessado].pop(0)
                            
                            #atualizando tempo que a funcional vai ficar ocupada
                            self.__ocupada[funcional] = minuto + tma
                            
                            #Verificando se a demanda foi atendida no prazo
                            #caso positivo podemos criar um novo dado
                            if self.__cdatetime + timedelta(minutes = minuto + tma) <= demanda_prazo[rd]:
                                sla_rd += 1 
                            
                            '''print('rd retirada: {} ocupada: {} prazo: {} skill: {}'.format(rd,
                                                                                 self.__ocupada[funcional],
                                                                                 self.sla[rd],
                                                                                 skill_acessado))'''
                            
                            #se for último elemento da etapa_rd, então excluo a chave
                            if len(etapa_rd[skill_acessado]) == 0:
                                etapa_rd.pop(skill_acessado)
                        
        #6. Retorno a quantidade de rds feitas no prazo para avaliar a qualidade do indivíduo
        return sla_rd


# Criando população inicial

In [ ]:
#ordem original -- não dever ser alterado
ordem = np.array(list(demanda.keys()), dtype=np.str)
pop_size = 50
pop = {}

for ii in range(pop_size):
    ordem_atendimento = ordem.copy()
    np.random.shuffle(ordem_atendimento)
    pop[ii] = list(ordem_atendimento)
    
    del ordem_atendimento

# Fit Function

A função para avaliar a qualidade dos indivíduos será a contagem de SLA perdida devido a ordem de seleção das demandas.

Toda demanda selecionada tem as seguintes características:
1. Tem de ser alocada em alguma funcional
2. A funcional precisa ficar um tempo com essa demanda (tempo médio de atendimento)

O resultado é influenciado pela ordem com que escolho as demandas. Cada indivíduo será um vetor que guarda para cada demanda a ordem em que cada demanda foi selecionada.

In [ ]:
resultado = {}
for ii in range(pop_size):
    g = grupo()

    for jj in range(10):
        skills_tma = {'etapa 1':15, 'etapa 2': 5, 'etapa 3': 3, 'etapa 4': 10, 'etapa 5': 30}
        g.funcional('000' + str(jj),skills_tma=skills_tma.copy())
    
        skills_tma = {'etapa 5':30, 'etapa 4': 10, 'etapa 3': 3, 'etapa 2': 5, 'etapa 1': 2}
        g.funcional('000' + str(jj), skills_tma=skills_tma.copy())

    resultado[ii] = g.atendimento(400, demanda_etapa.copy(),demanda_prazo.copy(), pop[ii].copy())

# Escolhendo os indivíduos mais promissores
Os indivíduos que possuem os melhores resultados são mantidos.

Os de menor qualidade são sorteados para que alguns sejam mantidos

In [ ]:
perc_melhores = 0.3
perc_piores = 0.1

ordenar_individuos = sorted(resultado.items(), key = lambda x:x[1], reverse=True)

#Guardando os resultados para comparativos futuros
#keep_results = ordenar_individuos[:]

#melhores indivíduos
best_individual = [i for i in ordenar_individuos[:int(perc_melhores*pop_size)]]

#sorteando os piores
worst_individual = [i[0] for i in ordenar_individuos[int(perc_melhores*pop_size):]]
rand_worst = list(np.random.choice(worst_individual, replace= False, size= int(perc_piores * len(worst_individual))))

rand_worst = [i for i in ordenar_individuos if i[0] in rand_worst]

#indivíduos para nova etapa
individuals = best_individual + rand_worst

del ordenar_individuos, best_individual, worst_individual, rand_worst

# Breeding

Criar filhos a partir do DNA dos pais

In [ ]:
#1.Identificar quem vai trocar material entre si
casais = np.random.choice([i[0] for i in individuals],replace=True,size=[pop_size,2])

#2.Regra de troca de DNA -- sorteio aleatório
for i, casal in enumerate(casais):
    
    #posso usar várias vezes o mesmo individuo para vários cruzamentos
    aux, breed= {}, []
    
    aux[casal[0]] = pop[casal[0]].copy()
    aux[casal[1]] = pop[casal[1]].copy()

    while len(aux[casal[0]]) and len(aux[casal[1]]):
        choice = np.random.choice([casal[0], casal[1]])

        #sempre descarto
        rd = aux[choice].pop(0)

        #se ainda não existe no gene do filho eu add, caso contrário descarto
        if not rd in breed:
            breed.append(rd)
    
    #crio a nova geração sobre a antiga
    pop[i] = breed[:]

# Mutation
A mutação acontece adicionando novos indivíduos a população. Não mudos os que já existem, simplesmente sorteio alguns e crio um filho com mutação. No caso atual mutação significa trocar algumas posições na ordem das demandas.

In [ ]:
perc_active_mutation = 0.1 #população que sofre mutação
perc_mutacao = 0.1 #percentual dos genes

sample = np.random.choice(range(pop_size), replace=False, size = int(pop_size*perc_active_mutation))

for choice in sample:
    
    length = len(pop[choice])
    changes = np.random.choice(range(length), replace = False, size = [int(length*perc_mutacao), 2])
    
    for change in changes:
        #invertendo os genes
        aux_1 = pop[choice][change[0]][:]
        aux_2 = pop[choice][change[1]][:]
        
        pop[choice][change[0]] = aux_2
        pop[choice][change[1]] = aux_1

In [ ]:
for geracao in range(300):
    start = time.time()
    
    resultado = {}
    for ii in range(pop_size):
        g = grupo()

        for jj in range(10):
            skills_tma = {'etapa 1':15, 'etapa 2': 5, 'etapa 3': 3, 'etapa 4': 10, 'etapa 5': 30}
            g.funcional('000' + str(jj),skills_tma=skills_tma.copy())

            skills_tma = {'etapa 5':30, 'etapa 4': 10, 'etapa 3': 3, 'etapa 2': 5, 'etapa 1': 2}
            g.funcional('000' + str(jj), skills_tma=skills_tma.copy())

        resultado[ii] = g.atendimento(400, demanda_etapa.copy(),demanda_prazo.copy(), pop[ii].copy())


    perc_melhores = 0.3
    perc_piores = 0.2

    ordenar_individuos = sorted(resultado.items(), key = lambda x:x[1], reverse=True)

    #Guardando os resultados para comparativos futuros
    #keep_results = ordenar_individuos[:]

    #melhores indivíduos
    best_individual = [i for i in ordenar_individuos[:int(perc_melhores*pop_size)]]

    #sorteando os piores
    worst_individual = [i[0] for i in ordenar_individuos[int(perc_melhores*pop_size):]]
    rand_worst = list(np.random.choice(worst_individual, replace= False, size= int(perc_piores * len(worst_individual))))

    rand_worst = [i for i in ordenar_individuos if i[0] in rand_worst]

    #indivíduos para nova etapa
    individuals = best_individual + rand_worst

    #del best_individual, worst_individual, rand_worst


    casais = np.random.choice([i[0] for i in individuals],replace=True,size=[pop_size,2])

    #2.Regra de troca de DNA -- sorteio aleatório
    for i, casal in enumerate(casais):

        #posso usar várias vezes o mesmo individuo para vários cruzamentos
        aux, breed= {}, []

        aux[casal[0]] = pop[casal[0]].copy()
        aux[casal[1]] = pop[casal[1]].copy()

        while len(aux[casal[0]]) and len(aux[casal[1]]):
            choice = np.random.choice([casal[0], casal[1]])

            #sempre descarto
            rd = aux[choice].pop(0)

            #se ainda não existe no gene do filho eu add, caso contrário descarto
            if not rd in breed:
                breed.append(rd)

        #crio a nova geração sobre a antiga
        pop[i] = breed[:]

    perc_active_mutation = 0.1 #população que sofre mutação
    perc_mutacao = 0.05 #percentual dos genes

    sample = np.random.choice(range(pop_size), replace=False, size = int(pop_size*perc_active_mutation))

    for choice in sample:

        length = len(pop[choice])
        changes = np.random.choice(range(length), replace = False, size = [int(length*perc_mutacao), 2])

        for change in changes:
            #invertendo os genes
            aux_1 = pop[choice][change[0]][:]
            aux_2 = pop[choice][change[1]][:]

            pop[choice][change[0]] = aux_2
            pop[choice][change[1]] = aux_1
    
    sys.stdout.write('\rGeracao: {} Tempo: {} Melhor: {} Pior: {}'.format(geracao,
                                                                          time.time() - start,
                                                                          ordenar_individuos[0],
                                                                          ordenar_individuos[1]))
    
    del ordenar_individuos